# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
file_one = "output_data/cities.csv"
cities_df = pd.read_csv(file_one)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,52.05,57,100,26.02,AU,1627350770
1,punta arenas,-53.1500,-70.9167,41.11,81,75,5.75,CL,1627350771
2,rikitea,-23.1203,-134.9692,74.75,80,98,17.38,PF,1627350771
3,vaini,-21.2000,-175.2000,77.16,83,75,11.50,TO,1627350771
4,torbay,47.6666,-52.7314,58.68,90,20,9.22,CA,1627350772


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [6]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
# cities_df = cities_df.dropna()
humidity_rate = cities_df["Humidity"].astype(float)

In [8]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 6)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# Ideal weather conditions
new_cities_df = cities_df.loc[(cities_df["Humidity"] <= 60) & (cities_df["Cloudiness"] > 70) & (cities_df["Wind Speed"] > 7), :]
new_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,52.05,57,100,26.02,AU,1627350770
34,new norfolk,-42.7826,147.0587,60.30,44,71,7.14,AU,1627350784
50,srednekolymsk,67.4500,153.6833,56.16,45,95,16.96,RU,1627350791
81,norman wells,65.2820,-126.8329,64.44,59,90,9.22,CA,1627350805
98,khatanga,71.9667,102.5000,70.00,54,100,19.17,RU,1627350812
144,baiyin,36.5583,104.2081,68.88,33,89,7.74,CN,1627350830
169,kabalo,-6.0500,26.9167,69.31,38,92,8.43,CD,1627350841
184,stepnogorsk,52.3544,71.8869,59.43,50,99,7.85,KZ,1627350847
199,port hedland,-20.3167,118.5667,77.05,53,87,14.97,AU,1627350853
202,muriti,-1.9833,32.9167,66.31,52,94,20.85,TZ,1627350854


In [16]:
# Drop any rows with null values
new_cities_df = new_cities_df.dropna()
new_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,52.05,57,100,26.02,AU,1627350770
34,new norfolk,-42.7826,147.0587,60.30,44,71,7.14,AU,1627350784
50,srednekolymsk,67.4500,153.6833,56.16,45,95,16.96,RU,1627350791
81,norman wells,65.2820,-126.8329,64.44,59,90,9.22,CA,1627350805
98,khatanga,71.9667,102.5000,70.00,54,100,19.17,RU,1627350812


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Create new dataframe with City, Country, Lat and Lng
hotel_df = new_cities_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df

,City,Country,Lat,Lng
0,busselton,AU,-33.6500,115.3333
34,new norfolk,AU,-42.7826,147.0587
50,srednekolymsk,RU,67.4500,153.6833
81,norman wells,CA,65.2820,-126.8329
98,khatanga,RU,71.9667,102.5000
144,baiyin,CN,36.5583,104.2081
169,kabalo,CD,-6.0500,26.9167
184,stepnogorsk,KZ,52.3544,71.8869
199,port hedland,AU,-20.3167,118.5667
202,muriti,TZ,-1.9833,32.9167


In [19]:
# Add Hotel Name column
hotel_df["Hotel Name"]= ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,busselton,AU,-33.6500,115.3333,
34,new norfolk,AU,-42.7826,147.0587,
50,srednekolymsk,RU,67.4500,153.6833,
81,norman wells,CA,65.2820,-126.8329,
98,khatanga,RU,71.9667,102.5000,
144,baiyin,CN,36.5583,104.2081,
169,kabalo,CD,-6.0500,26.9167,
184,stepnogorsk,KZ,52.3544,71.8869,
199,port hedland,AU,-20.3167,118.5667,
202,muriti,TZ,-1.9833,32.9167,


In [38]:


for idx,row in hotel_df.iterrows():
#     print(row["Lat"])
    Lat = row["Lat"]
    Lng = row["Lng"]
    # geocoordinates
    target_coordinates = f"{Lat},{Lng}"
#     target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
#         "keyword": target_search,
        "radius": target_radius,
        "types": target_type,
        "key": g_key
    }

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    try:
        hotel_df.loc[idx,"Hotel Name"]=(response.json()["results"][0]["name"])
    except:
        print("No hotel found here")

hotel_df
    

No hotel found here
No hotel found here
No hotel found here
No hotel found here
No hotel found here
No hotel found here
No hotel found here


,City,Country,Lat,Lng,Hotel Name
0,busselton,AU,-33.6500,115.3333,Observatory Guest House
34,new norfolk,AU,-42.7826,147.0587,The Shingles Riverside Cottages
50,srednekolymsk,RU,67.4500,153.6833,Srednekolymsk
81,norman wells,CA,65.2820,-126.8329,Heritage Hotel
98,khatanga,RU,71.9667,102.5000,"Gostinitsa ""Zapolyar'ye"""
144,baiyin,CN,36.5583,104.2081,GreenTree Alliance Gansu Baiyin Renmin Road Hotel
169,kabalo,CD,-6.0500,26.9167,Kabalo
184,stepnogorsk,KZ,52.3544,71.8869,Gostinitsa Stepnogorsk
199,port hedland,AU,-20.3167,118.5667,The Esplanade Hotel
202,muriti,TZ,-1.9833,32.9167,Muriti


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [43]:
# Add marker layer ontop of heat map
marker_layer = gmaps.symbol_layer(
    locations, fill_color = 'rgba(0,150,0,0.4)',
    stroke_color = 'rgba(0,0,150,0.4)', scale = 2,
    info_box_content = [f"{hotel_name}" for hotel_name in hotel_df["Hotel Name"]]
)

# Display figure
fig = gmaps.figure()
fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [44]:
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))